# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Необходимо построить модель с максимально большим значением *accuracy*. Требованием заказчика является значение *accuracy* модели не менее 0.75. *Accuracy* будет считаться на тестовой выборке.

## Подготовка данных

Импортируем необходимые библиотеки и методы.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Загрузим данные в переменную `data`.

In [2]:
data = pd.read_csv('users_behavior.csv')

Изучим данные методами `head()`, `describe()` и `info()`.

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбитие данных на выборки

Запишем признаки в переменную `features`, а целевой признак в переменную `target`.

In [6]:
target = data['is_ultra']
features = data.drop('is_ultra', axis=1)

Разобьём данные на 4 выборки в соотношении 3:1:1. 60% данных уйдёт на тренировочную выборку, и по 20% на валидационную и тестовую.

In [7]:
#Сначала разобьём обучающие признаки и целевой признак на тренировочную и тестовую выборки. 
#Поместим 20% данных в тестовую выборку, а 80% оставим в обучающей.
train_features, test_features, train_target, test_target = \
                train_test_split(features, target, test_size=0.2, train_size=0.8, random_state=322)

In [8]:
#Разобьём повторно тренировочную выборку на две, в соотношении 3:1. 
#Таким образом, в тренировочных данных останется 60% данных, а в валидационной - 20%.
train_features, valid_features, train_target, valid_target = \
    train_test_split(train_features, train_target, test_size=0.25, train_size=0.75, random_state=322)

Проверим процент данных в выборках:

In [9]:
print('Процент данных в обучающей выборке:', 
      round(train_features.shape[0] / features.shape[0] * 100), '%')
print('Процент данных в тестовой выборке:', 
      round(test_features.shape[0] / features.shape[0] * 100), '%')
print('Процент данных в валидационной выборке:', 
      round(valid_features.shape[0] / features.shape[0] * 100), '%')

Процент данных в обучающей выборке: 60 %
Процент данных в тестовой выборке: 20 %
Процент данных в валидационной выборке: 20 %


Данные разбиты корректно.

## Исследование моделей

### `DecisionTreeClassifier`

Сначала обучим модель классификационного дерева. Будем изменять гиперпараметр максимальной глубины дерева. Сравним точность модели с гиперпараметром `max_depth` в диапазоне от 1 до 10 на валидационной выборке.

In [10]:
best_tree_model = None
best_tree_result = 0
best_models_accuracy_score = {}

for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=322, max_depth=depth)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    result = accuracy_score(valid_target, predictions)
    if result > best_tree_result:
        best_tree_result = result
        best_tree_model = model
        best_models_accuracy_score['Tree'] = result
        
print('Точность лучшей модели "дерева" на валидационной выборке:', best_tree_result)
print('Глубина лучшей модели на валидационной выборке:', best_tree_model.max_depth)

Точность лучшей модели "дерева" на валидационной выборке: 0.8102643856920684
Глубина лучшей модели на валидационной выборке: 4


Лучшая модель имеет точность 0.81, а её глубина составляет 4.

### `RandomForestClassifier`

Обучим и проверим на валидационной выборке модель случайного леса. В качестве гиперпараметра выберем количество деревьев. Будем изменять его в диапазоне от 10 до 100, с шагом в 10.

In [11]:
best_forest_model = None
best_forest_result = 0

for est in range(10, 101, 10):
    model = RandomForestClassifier(random_state=322, n_estimators=est)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    result = accuracy_score(valid_target, predictions)
    if result > best_forest_result:
        best_forest_result = result
        best_forest_model = model
        best_models_accuracy_score['Forest'] = result
        
print('Точность лучшей модели "леса" на валидационной выборке:', best_forest_result)
print('Количество деревьев в лучшей модели на валидационной выборке:', best_forest_model.n_estimators)

Точность лучшей модели "леса" на валидационной выборке: 0.8195956454121306
Количество деревьев в лучшей модели на валидационной выборке: 30


Точность на валидационной выборке получилась немного лучше, чем при использовании одиночного дерева. Количество деревьев в лучшей модели составило 30.

### `LogisticRegression`

Также, проверим модель логистической регрессии. В качестве гиперпараметра выберем количество итераций. Будем изменять его от 100 до 10000, с шагом в 100.

In [12]:
best_logistic_model = None
best_logistic_result = 0

for iterations in range(100, 10001, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iterations)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    result = accuracy_score(valid_target, predictions)
    if result > best_logistic_result:
        best_logistic_result = result
        best_logistic_model = model
        best_models_accuracy_score['Logistic regression'] = result
        
print('Точность лучшей модели логистической регрессии на валидационной выборке:', best_logistic_result)
print('Количество итераций в лучшей модели на валидационной выборке:', best_logistic_model.max_iter)

Точность лучшей модели логистической регрессии на валидационной выборке: 0.6920684292379471
Количество итераций в лучшей модели на валидационной выборке: 100


Данная модель проявила себя хуже всего, точность не проходит даже порог, установленный в задании: 0.75.

Для наглядности составим `Series` со значениями лучшей точности для каждой из моделей: 

In [13]:
accuracy_series = pd.Series(index=best_models_accuracy_score.keys(), 
                            data=best_models_accuracy_score.values()).sort_values(ascending=False)
accuracy_series

Forest                 0.819596
Tree                   0.810264
Logistic regression    0.692068
dtype: float64

Лучшей моделью на валидационной выборке оказалась модель случайного леса с 30 деревьями. У неё преимущество в точности над лучшей моделью одиночного дерева меньше 1%, но преимущество всё равно есть. Поэтому, для тестовой выборки выбираем модель случайного леса с 30 деревьями.

## Проверка модели на тестовой выборке

Проверим работу модели на тестовой выборке:

In [14]:
print('Точность лучшей модели "леса" на тестовой выборке:', 
      accuracy_score(test_target, best_forest_model.predict(test_features)))

Точность лучшей модели "леса" на тестовой выборке: 0.7993779160186625


Точность нашей лучшей модели немного снизилась по сравнению с валидационной выборкой, но незначительно, модель всё равно удовлетворяет условиям задачи.

## Вывод

В ходе выполнения проекта были обучены модели трёх видов, и выбрана лучшая из них, с помощью изменения значений гиперпараметров. Лучшей моделью на тестовой выборке оказалась модель случайного леса с 30 деревьями. Она удовлетворяет условиям задачи, её точность составляет почти 0.8.